TODO
* ~Merge Tables 2,4,10 to work together~
* ~Create loop for full season~
* Add season and episode to front of data frame

Someday:
* Fix Italics:
    * Work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
    * Not work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
        
Lessons Learned: 
* Look for specific class types in BeautifulSoup. The fact that the transcript was formated as a *wikitable* made life much easier
* Edge Cases

In [1]:
# imports

#standard
import pandas as pd
#import random
from datetime import datetime
from pandasql import sqldf
pysqldf = lambda q: sqldf(q,globals())

#Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Data Pulls
from bs4 import BeautifulSoup
import requests
import re

In [2]:
#Base URL for Transcripts
url_base = 'https://avatar.fandom.com/wiki/Avatar_Wiki:Transcripts'
response = requests.get(url_base)

#Count # of Tables of Transcript Page
soup = BeautifulSoup(response.content, "html.parser")
tables = soup.find_all("table")
print(f"Found {len(tables)} tables on the page.")

# Table 2: Book 1 Water
# Table 4: Book 2 Earth
# Table 10: Book 3 Fire

# if len(tables) > 0:
#     print("First table:")
#     print(tables[10].prettify())

Found 90 tables on the page.


In [3]:
book_list = [2, #Water
             5, #Earth
             10] # Fire

episode_list = []

for book in book_list:
    specific_table = soup.find_all("table")[book]  # Replace 0 with the index of the table you want
    links = specific_table.find_all("a")
    for link in links:
        episode_list.append(link.get("href"))

In [4]:
#Remove url links for commentary links
episode_list_filtered = [episode for episode in episode_list if "commentary" not in episode]
#episode_list_filtered

#Creates Episodes URLs
base = 'https://avatar.fandom.com'
url_episode_list = []
for episode in episode_list_filtered:
    #print(base + episode)
    url_episode_list.append(base + episode)

### Edge Case Identifier

In [ ]:
#Printe Edge Case Episodes
for episode_number, url in enumerate(url_episode_list):
    
    split_text = url.split(':', 2)
    episode_title = split_text[2]

    episode_number += 1
    # print(episode_number)
    # print(episode_title)
    # print(url)
    
    response = requests.get(url)
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the tables with the class "wikitable"
    wikitable_tables = soup.find_all("table", class_="wikitable")
    num_tables = len(wikitable_tables)
    
    if num_tables >= 2:
            print(episode_number)
            print(episode_title)
            print(url)
            print(f"Number of Tables: {len(wikitable_tables)}")

In [ ]:
# Edge Case Exploration

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg'
# Index 1 | 2nd Table

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns'
# Index 1 2nd Table 

# url = 'https://avatar.fandom.com/wiki/Transcript:Bato_of_the_Water_Tribe'
# Index 0 1st Table 
# 2nd Table contains a deleted scene

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Tales_of_Ba_Sing_Se'
# Index 0 The Tale of Toph and Katara
# Index 1 The Tale of Iroh
# Index 2 The Tale of Aang
# Index 3 The Tale of Sokka 
# Index 4 THe Tale of Zuko

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Puppetmaster'
# Index 1 2nd Table 
# Removes the "previously on Avatar"
    
response = requests.get(url)
ep_soup = BeautifulSoup(response.content, "html.parser")
tables = ep_soup.select("table.wikitable")

data = []

print(f"Number of Tables: {len(tables)}")
print()

desired_table = tables[:4]  
# Find all rows within the 2nd Table that holds the transcript
rows = desired_table.select("tbody tr")

for row in rows:
    character_cell = row.find('th')  # Find the <th> for character names
    dialogue_cell = row.find('td')    # Find the <td> for character's dialogue

    if character_cell and dialogue_cell:  # Ensure both character & dialogue are present (thus skipping pure screen directions)
        character = character_cell.get_text(strip=True)

        # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
        for italics in dialogue_cell.find_all('i'):
            # Get the text inside <i>, append '123' and replace the <i> with this text
            italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
            italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

        dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

        data.append({'character': character, 'dialogue': dialogue})

        df = pd.DataFrame(data)

#Cleaning
df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

#Word Count
df['word_count'] = df['dialogue'].str.split().apply(len)

df

In [ ]:
#Tales of Ba Sing Se - Edge Case Exploration
url = 'https://avatar.fandom.com/wiki/Transcript:The_Tales_of_Ba_Sing_Se'
# Index 0 The Tale of Toph and Katara
# Index 1 The Tale of Iroh
# Index 2 The Tale of Aang
# Index 3 The Tale of Sokka 
# Index 4 THe Tale of Zuko
    
response = requests.get(url)
ep_soup = BeautifulSoup(response.content, "html.parser")
tables = ep_soup.select("table.wikitable")

data = []

# Iterate over the first 5 tables (0 to 4)
for table in tables[:5]:  
    # Find all rows within each table
    rows = table.select("tbody tr")

    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
            character = character_cell.get_text(strip=True)

            # Process the <i> tags within the dialogue
            for italics in dialogue_cell.find_all('i'):
                # Prepend and append '12345' to italicized text
                italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

            # Get the full dialogue text after modifications
            dialogue = dialogue_cell.get_text(strip=True)

            # Append the data to the list
            data.append({'character': character, 'dialogue': dialogue})

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

#Cleaning
df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

#Word Count
df['word_count'] = df['dialogue'].str.split().apply(len)

pd.set_option('display.max_rows', None)

df

### Full Series Trancript Pull
* Includes all Non Edge Cases
* All edge cases including:
    * Where primary table is index 1 (rather than index 0) 
    * Tales of Ba Sing Se

In [5]:
#episodes, so technically need to subtract 1 from each to get the edge case
edge_case_episodes = [1, #The_Boy_in_the_Iceberg
                      2, #The_Avatar_Returns
                      #15, #Bato_of_the_Water_Tribe - Use table 1, index 0 to remove deleted scence
                      35, #The_Tales_of_Ba_Sing_Se
                      48 #The_Puppetmaster
                     ]

#Subtract 1 from each episode to convert episode_number --> index
edge_case_episodes = [episode - 1 for episode in edge_case_episodes]
#print(edge_case_episodes)

In [6]:
transcript_df = pd.DataFrame()

for episode_number, url in enumerate(url_episode_list):
    # Splits based on Edge Cases
    
    data = []  # Initialize data list for an individual episode
    
    # If Statement splits data into base and non-edge cases
    if episode_number not in edge_case_episodes:
        
        response = requests.get(url)
        ep_soup = BeautifulSoup(response.content, "html.parser")
        tables = ep_soup.select("table.wikitable")
        
        desired_table = tables[0]
        rows = desired_table.select("tbody tr")

    if episode_number in edge_case_episodes:
            
            if episode_number != 34:
                response = requests.get(url)
                ep_soup = BeautifulSoup(response.content, "html.parser")
                tables = ep_soup.select("table.wikitable")
                
                desired_table = tables[1]
                rows = desired_table.select("tbody tr")
                
            else:
                response = requests.get(url)
                ep_soup = BeautifulSoup(response.content, "html.parser")
                tables = ep_soup.select("table.wikitable")
                
                all_rows = []  # Initialize list to store all rows from all tables

                for table in tables[:5]:
                    table_rows = table.select("tbody tr")
                    all_rows.extend(table_rows)  # Append all rows to the all_rows list
                    
                rows = all_rows  # Now, 'rows' contains all the rows from tables 0 to 4
                    
    split_text = url.split(':', 2)
    episode_title = split_text[2]
    episode_number += 1
    
    # print(episode_number)
    # print(episode_title)
    # print(url)
    # print()

    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (handle italicized text)
            for italics in dialogue_cell.find_all('i'):
                italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics
            data.append({'character': character, 'dialogue': dialogue})

    # Create DataFrame if the data was extracted
    if data:
        df = pd.DataFrame(data)

        # Cleaning the dialogue column
        df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()  # Remove text in []
        df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()  # Remove text in ()
        df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()  # Replace '12345' with space

        # Add word count
        df['word_count'] = df['dialogue'].str.split().apply(len)

        # Add episode number and title
        df['episode'] = episode_number
        df['episode_title'] = episode_title 

        # Concatenate with the main transcript DataFrame
        transcript_df = pd.concat([transcript_df, df], ignore_index=True)

1
The_Boy_in_the_Iceberg
https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg

2
The_Avatar_Returns
https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns

3
The_Southern_Air_Temple
https://avatar.fandom.com/wiki/Transcript:The_Southern_Air_Temple

4
The_Warriors_of_Kyoshi
https://avatar.fandom.com/wiki/Transcript:The_Warriors_of_Kyoshi

5
The_King_of_Omashu
https://avatar.fandom.com/wiki/Transcript:The_King_of_Omashu

6
Imprisoned
https://avatar.fandom.com/wiki/Transcript:Imprisoned

7
Winter_Solstice,_Part_1:_The_Spirit_World
https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_1:_The_Spirit_World

8
Winter_Solstice,_Part_2:_Avatar_Roku
https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_2:_Avatar_Roku

9
The_Waterbending_Scroll
https://avatar.fandom.com/wiki/Transcript:The_Waterbending_Scroll

10
Jet_(episode)
https://avatar.fandom.com/wiki/Transcript:Jet_(episode)

11
The_Great_Divide
https://avatar.fandom.com/wiki/Transcript:The_Great_Divid

In [ ]:
#Edge Case Check - All Working

In [7]:
transcript_df[transcript_df['episode'] ==2].head(126)

,character,dialogue,word_count,episode,episode_title
153,Village kids,Yay! Aang's back!,3,2,The_Avatar_Returns
154,Sokka,I knew it! You signaled the Fire Navy with th...,19,2,The_Avatar_Returns
155,Katara,Aang didn't do anything! It was an accident.,8,2,The_Avatar_Returns
156,Aang,Yeah. We were on the ship and there was this b...,21,2,The_Avatar_Returns
157,Kanna,"Katara, you shouldn't have gone on that ship! ...",15,2,The_Avatar_Returns
...,...,...,...,...,...
272,Sokka,I'd like that. I'd reeally like that.,7,2,The_Avatar_Returns
273,Katara,Then we're in this together!,5,2,The_Avatar_Returns
274,Aang,"All right, but before I learn waterbending, we...",19,2,The_Avatar_Returns
275,Katara,What's there?,2,2,The_Avatar_Returns


In [8]:
transcript_df[transcript_df['episode'] ==35].head(126)

,character,dialogue,word_count,episode,episode_title
5827,Katara,Toph! Aren't you gonna get ready for the day?,9,35,The_Tales_of_Ba_Sing_Se
5828,Toph,I'm ready.,2,35,The_Tales_of_Ba_Sing_Se
5829,Katara,You're not gonna wash up? You've got a little ...,15,35,The_Tales_of_Ba_Sing_Se
5830,Toph,"You call it dirt, I call it a healthy coating ...",12,35,The_Tales_of_Ba_Sing_Se
5831,Katara,Hmm ... You know what we need? A girl's day out!,11,35,The_Tales_of_Ba_Sing_Se
...,...,...,...,...,...
5948,Jin,Your uncle is a good teacher. I have somethin...,20,35,The_Tales_of_Ba_Sing_Se
5949,Jin,What's wrong?,2,35,The_Tales_of_Ba_Sing_Se
5950,Zuko,It's complicated. I have to go.,6,35,The_Tales_of_Ba_Sing_Se
5951,Iroh,"How was your night, Prince Zuko?",6,35,The_Tales_of_Ba_Sing_Se


In [9]:
transcript_df[transcript_df['episode'] ==48].tail()

,character,dialogue,word_count,episode,episode_title
7930,Aang,It's okay!,2,48,The_Puppetmaster
7931,Hama,"Don't hurt your friends, Katara! And don't let...",12,48,The_Puppetmaster
7932,Katara,No!,1,48,The_Puppetmaster
7933,Male prisoner #1,You're going to be locked away forever.,7,48,The_Puppetmaster
7934,Hama,"My work is done. Congratulations, Katara. You...",9,48,The_Puppetmaster
